In [70]:
from dataloader import CarlaDataset, Instance
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_sequence, unpack_sequence, pad_sequence, pad_packed_sequence
import torch.nn.functional as F
from copy import copy, deepcopy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
trainset = CarlaDataset.load_dataset("../datasets/train1")
validaionset = CarlaDataset.load_dataset("../datasets/train2")

In [78]:
instances = [instance for instance in trainset]
a = deepcopy(instances[0])
a.gcp_pv += 1
a.gcp_pv

array([268, 526])

In [44]:
def my_collate(batch: list[Instance]):
    data = [torch.tensor(item.hull_pv, dtype=torch.float32) for item in batch]
    data = torch.nested.nested_tensor(data)
    targets = torch.tensor([item.gcp_pv for item in batch])
    return (data, targets)

In [45]:
train_loader = DataLoader(trainset, batch_size=10, shuffle=True, collate_fn=my_collate)
validation_loader = DataLoader(validaionset, batch_size=10, shuffle=True, collate_fn=my_collate)

In [46]:
batch_hulls, batch_gcps = next(iter(train_loader))
batch_gcps.shape

torch.Size([10, 2])

In [5]:
gcps = np.array([instance[1] for instance in trainset])
hulls = [instance[0] for instance in trainset]
hulls = [torch.tensor(hull, dtype=torch.float32) for hull in hulls]

gcps = torch.tensor(gcps, dtype=torch.float32)
nested_hulls = torch.nested.nested_tensor(hulls, dtype=torch.float32)
padded_hulls = torch.nested.to_padded_tensor(nested_hulls, padding=0.0)
#hulls = pad_sequence(hulls, batch_first=True)

/tmp/ipykernel_1411645/2766378237.py:6: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/aten/src/ATen/NestedTensorImpl.cpp:178.)
  nested_hulls = torch.nested.nested_tensor(hulls, dtype=torch.float32)


In [6]:
special_token = torch.rand(3, 2, requires_grad=True)
special_token = special_token.broadcast_to((615, 3, 2))
special_token.shape

torch.Size([615, 3, 2])

In [7]:
torch.cat((special_token, padded_hulls), dim=1).shape

torch.Size([615, 43, 2])

In [8]:
encoder_layer = nn.TransformerEncoderLayer(d_model=2, nhead=2, batch_first=True)
encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

In [9]:
decoder_layer = nn.TransformerDecoderLayer(d_model=2, nhead=2, batch_first=True)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)

In [10]:
output = encoder_layer(padded_hulls)
output.shape

torch.Size([615, 40, 2])

In [11]:
rnn = nn.RNN(2, 32, 2, batch_first=True)

In [12]:
out, hidden = rnn(hulls)
print(type(out), type(hidden))
hidden.shape

AttributeError: 'list' object has no attribute 'dim'

In [ ]:
test, lens = pad_packed_sequence(out, batch_first=True)
test.shape

NameError: name 'out' is not defined

In [ ]:
out[4].shape

NameError: name 'out' is not defined

In [ ]:
pooled = test.max(dim=1).values
pooled.shape

NameError: name 'test' is not defined

In [ ]:
out[1].shape

NameError: name 'out' is not defined

In [ ]:
from typing import Literal
from torch.nn.utils.rnn import PackedSequence


INPUT_SIZE = 2 # 2D points for each hull coordinate
GCP_SIZE = 2 # 2D points for each GCP coordinate


class GcpRegressor(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(GcpRegressor, self).__init__()
        self.rnn = nn.RNN(INPUT_SIZE, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, GCP_SIZE)

    def forward(self, x: PackedSequence):
        """_summary_

        Args:
            x (_type_): _description_

        Returns:
            _type_: _description_
        """
        out, _hidden = self.rnn(x)
        out = pad_sequence(out, batch_first=True) # shape (batch_size, max_hull_len, hidden_size)
        out = out.max(dim=1).values          
        out = self.linear(out)
        return out

In [ ]:
test = torch.rand(100, 40, 32)
pooled = torch.nn.functional.max_pool1d(test, 32).squeeze()
pooled.shape

torch.Size([100, 40])

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    instances = torch.nested.nested_tensor([instance[0] for instance in batch], dtype=torch.float32).to_padded_tensor(0)
    targets = torch.tensor([instance[1] for instance in batch], dtype=torch.float32)
    
    return instances, targets

train_loader = DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(validaionset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [ ]:
import pytorch_lightning as pl
from typing import Any, Literal
from pytorch_lightning.utilities.types import STEP_OUTPUT, OptimizerLRScheduler
from scipy import optimize
from torch.nn.utils.rnn import PackedSequence


INPUT_SIZE = 2 # 2D points for each hull coordinate
GCP_SIZE = 2 # 2D points for each GCP coordinate


class GcpRegressor(pl.LightningModule):
    def __init__(self, hidden_size, num_layers):
        super(GcpRegressor, self).__init__()
        self.rnn = nn.RNN(INPUT_SIZE, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, GCP_SIZE)

    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        x, y = batch
        rnn_output, _hidden = self.rnn(x)
        pooled = rnn_output.max(dim=1).values
        linear = self.linear(pooled)
        predictions = nn.functional.sigmoid(linear) # scale to [0, 1]
        loss = nn.functional.mse_loss(predictions, y)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self) -> OptimizerLRScheduler:
        optimizer = torch.optim.Adam(self.parameters(), lr=1)
        return optimizer


In [ ]:
gcp_regressor = GcpRegressor(32, 1)

In [ ]:
trainer = pl.Trainer(max_epochs=500)
trainer.fit(gcp_regressor, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params
----------------------------------
0 | rnn    | RNN    | 1.2 K 
1 | linear | Linear | 66    
----------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.


In [ ]:
class TestTransform(torch.nn.Module):
    def __init__(self, scaler):
        super(TestTransform, self).__init__()
        self.scaler = scaler
    
    def forward(self, x):
        x["a"] += self.scaler
        x["b"] += self.scaler
        return x
    
t = TestTransform(1)
x = {"a": 1, "b": 2}
t(x), x

({'a': 2, 'b': 3}, {'a': 2, 'b': 3})

In [ ]:
torch.__version__

'2.0.1+cu117'